<a href="https://colab.research.google.com/github/crystaljwang/tm10007_group_3/blob/import_data/ML_Crystal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The code below loads the GIST data from GitHub.

In [6]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

SyntaxError: invalid syntax (3447734670.py, line 2)

In [ ]:
# Run this to use from colab environment
!git clone https://github.com/jveenland/tm10007_ml.git

SyntaxError: invalid syntax (3994390392.py, line 2)

  DEPRECATION: ktml is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [ ]:
%cd /content/tm10007_ml/worcgist

/content/tm10007_ml/worcgist


In [ ]:
# ----- IMPORT LIBRARIES -----
from pathlib import Path
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
dir = Path('.') / 'GIST_radiomicFeatures.csv'
data = pd.read_csv(dir, index_col=0)

print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

The number of samples: 246
The number of columns: 494


In [ ]:

# ----- SPLITTING THE DATA -----

# Replace label values from string to binary
data['label'] = data['label'].replace({'GIST': 1, 'non-GIST': 0})

# Separate the features and labels
X = data.drop(['label'], axis=1)
y = data['label']

# Split the data into random train and test sets
X_train_tot, X_test, y_train_tot, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_tot, y_train_tot, test_size=0.15, random_state=42)
print(X_train.shape)
print(X_val.shape)

In [ ]:
# ----- OUTLIERS -----

def replace_outliers(data):
    """
    Replaces the outliers in a DataFrame with the lower or upper bound.

    :param data: The DataFrame to be filtered
    :return: A new DataFrame with the outliers replaced by the lower or upper bound for each column
    """
    # Calculate the lower and upper bounds based on each column's median and interquartile range
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Replace the outliers with the lower or upper bound
    for col in data.columns:
        data[col] = data[col].apply(lambda x: upper_bound[col] if x > upper_bound[col] else x)
        data[col] = data[col].apply(lambda x: lower_bound[col] if x < lower_bound[col] else x)

    return data

# Replace the outliers in each column with the lower or upper bound
replace_outliers(X_train)

# sns.boxplot(replaced_df[5])

In [ ]:
# Data scaling
scaler = MinMaxScaler()  # define scaler
# scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)  # fit scaler on train set
X_val = scaler.transform(X_val)  # apply fitted scaler on validation set
X_test = scaler.transform(X_test)  # apply fitted scaler on test set

In [ ]:
# Remove all constant (zero-variance) features
X_train = pd.DataFrame(X_train)
zero_var_filter = VarianceThreshold(threshold=0)

# Fit on train data
zero_var_filter.fit(X_train)
zero_var_columns = [column for column in X_train.columns if column not in X_train.columns[zero_var_filter.get_support()]]

# Apply on validation and test data
X_train = zero_var_filter.transform(X_train)
X_val = zero_var_filter.transform(X_val)
X_test = zero_var_filter.transform(X_test)

In [ ]:
# ----- Feature selection -----

from sklearn.linear_model import Lasso

lasso_selector = SelectFromModel(estimator=Lasso(alpha=10**(-10), max_iter=1000), threshold='median')
lasso_selector.fit(X_train, y_train)
lasso_list = [column for column in pd.DataFrame(X_train).columns[lasso_selector.get_support()]]
n_original = X_train.shape[1]

X_train = selector.transform(X_train)
n_selected = X_train.shape[1]
print(f"Selected {n_selected} from {n_original} features.")

In [ ]:
X_train.shape

In [ ]:
# ----- Feature extraction -----

from sklearn.decomposition import PCA

# 95% variance
pca = PCA(n_components = 0.95)
pca.fit(X_train)
X_train = pca.transform(X_train)

print(f"Selected {X_train.shape[1]} features to be used for classification.")